# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.


* *сalls*— количество звонков,
* *minutes* — суммарная длительность звонков в минутах,
* *messages* — количество sms-сообщений,
* *mb_used* — израсходованный интернет-трафик в Мб,
* *is_ultra* — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from joblib import dump

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
df.duplicated().sum()

0

In [7]:
df[['calls','messages']] = df[['calls','messages']].astype(int)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


In [9]:
df.shape

(3214, 5)

**Вывод**

Пропуска данных и дупликатов в датафрейме не обнаружено. Столбцы `calls` и `messages` были переведены в целочисленный формат.

## Разбейте данные на выборки

`df_train` - обучающая выборка

`df_valid` - валидационная выборка

`df_test` - тестовая выборка

In [10]:
df_train, df_valid = train_test_split(df, test_size=.25, random_state=12345)
df_test, df_valid = train_test_split(df_valid, test_size=.5, random_state=12345)
print('Размер обучающей выборки:', df_train.shape) # 75% data sets
print('Размер тестовой выборки:', df_test.shape) # 12.5% datasets
print('Размер валидационной выборки:', df_valid.shape) #12.5# data sets

Размер обучающей выборки: (2410, 5)
Размер тестовой выборки: (402, 5)
Размер валидационной выборки: (402, 5)


Для обучающей выборки:

In [11]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

Для валидационной выборки:

In [12]:
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

Для тестовой выборки:

In [13]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

## Исследуйте модели

Для начала, проверим константную модель на адекватность. За константу примем пример 0, так как это значение встречается 80% случаев

In [14]:
target_pred_constant = pd.Series(0, index=target_valid.index)
print(accuracy_score(target_valid, target_pred_constant)) 

0.7039800995024875


Воспользуемся тремя основными моделями: дерево решения, случайный лес и логистическая регрессия.

### Дерево решения

In [15]:
%%time

best_model_dr = None
best_result_dr = 0
best_depth = 0
for depth in range (1,50,2):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    prediction = model.predict(features_valid)
    result = accuracy_score(target_valid, prediction)
    if result > best_result_dr:
        best_model_dr = model
        best_result_dr = result
        best_depth = depth
    print('Глубина дерева:', depth, 'Качество модели:', result)
print('')
print("Лучший результат качества модели:", best_result_dr, 'при глубине:', best_depth)

Глубина дерева: 1 Качество модели: 0.736318407960199
Глубина дерева: 3 Качество модели: 0.7786069651741293
Глубина дерева: 5 Качество модели: 0.763681592039801
Глубина дерева: 7 Качество модели: 0.7910447761194029
Глубина дерева: 9 Качество модели: 0.7985074626865671
Глубина дерева: 11 Качество модели: 0.7960199004975125
Глубина дерева: 13 Качество модели: 0.7786069651741293
Глубина дерева: 15 Качество модели: 0.7711442786069652
Глубина дерева: 17 Качество модели: 0.7437810945273632
Глубина дерева: 19 Качество модели: 0.7189054726368159
Глубина дерева: 21 Качество модели: 0.7263681592039801
Глубина дерева: 23 Качество модели: 0.7189054726368159
Глубина дерева: 25 Качество модели: 0.7213930348258707
Глубина дерева: 27 Качество модели: 0.7213930348258707
Глубина дерева: 29 Качество модели: 0.7213930348258707
Глубина дерева: 31 Качество модели: 0.7213930348258707
Глубина дерева: 33 Качество модели: 0.7213930348258707
Глубина дерева: 35 Качество модели: 0.7213930348258707
Глубина дерева: 3

#### Дерево решений с разными гиперпараметрами

Так как модель показала наилучший результат при глубине: 9, то будем использовать это значение глубины и попробуй изменить гиперпараметры и посмотреть разницу точности.

##### Гиперпараметр `criterion='entropy'`

Попробуем поменять гиперпараметр `criterion`, который отвечает за измерение качества разделения. 

In [16]:
model = DecisionTreeClassifier(random_state=12345, max_depth=9, criterion='entropy')
model.fit(features_train, target_train)
prediction = model.predict(features_valid)
result = accuracy_score(target_valid, prediction)
    
print("Лучший результат качества модели:", result, 'при глубине:', 9)

Лучший результат качества модели: 0.7810945273631841 при глубине: 9


Гиперпараметр `criterion='entropy'` показал хуже результат точности модели, чем `criterion='gini'`

##### Гиперпаметр `splitter='random'`

Попробуем поменять гиперпараметр `splitter`, который отвечает за стратегию, используемую для выбора разделения в каждом узле.

In [17]:
model = DecisionTreeClassifier(random_state=12345, max_depth=9, splitter = 'random')
model.fit(features_train, target_train)
prediction = model.predict(features_valid)
result = accuracy_score(target_valid, prediction)
    
print("Лучший результат качества модели:", result, 'при глубине:', 9)

Лучший результат качества модели: 0.7611940298507462 при глубине: 9


Гиперпараметр `splitter='random'` показал хуже результат точности модели, чем `splitter='best'`

### Случайный лес

In [18]:
%%time

best_model_sl = None
best_result_sl = 0
best_est = 0
for est in range(1, 30):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result_sl:
        best_model_sl = model
        best_result_sl = result
        best_est = est
    print('Число деревьев:', est, 'Качество модели', result)
print('')
print("Лучший результат качества модели:", best_result_sl, 'при количестве деревев:', best_est)

Число деревьев: 1 Качество модели 0.753731343283582
Число деревьев: 2 Качество модели 0.7686567164179104
Число деревьев: 3 Качество модели 0.7562189054726368
Число деревьев: 4 Качество модели 0.7711442786069652
Число деревьев: 5 Качество модели 0.7686567164179104
Число деревьев: 6 Качество модели 0.7786069651741293
Число деревьев: 7 Качество модели 0.7686567164179104
Число деревьев: 8 Качество модели 0.7761194029850746
Число деревьев: 9 Качество модели 0.7736318407960199
Число деревьев: 10 Качество модели 0.7835820895522388
Число деревьев: 11 Качество модели 0.7810945273631841
Число деревьев: 12 Качество модели 0.7885572139303483
Число деревьев: 13 Качество модели 0.7736318407960199
Число деревьев: 14 Качество модели 0.7810945273631841
Число деревьев: 15 Качество модели 0.7835820895522388
Число деревьев: 16 Качество модели 0.7835820895522388
Число деревьев: 17 Качество модели 0.7736318407960199
Число деревьев: 18 Качество модели 0.7810945273631841
Число деревьев: 19 Качество модели 0.7

### Логистическая регрессия

In [19]:
%%time

best_model_lr = model
best_result_lr = 0

for itera in range (1,101):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=itera)
    model.fit(features_train, target_train)
    prediction = model.predict(features_valid)
    result = accuracy_score(target_valid, prediction)
    if result > best_result_lr:
        best_model_lr = model
        best_result_lr = result
    print('Номер итерации:', itera, 'Качество модели', result)

print("Лучший результат качества модели:", best_result_lr, 'при количестве итераций:', itera)

Номер итерации: 1 Качество модели 0.7039800995024875
Номер итерации: 2 Качество модели 0.7039800995024875
Номер итерации: 3 Качество модели 0.7039800995024875
Номер итерации: 4 Качество модели 0.7039800995024875
Номер итерации: 5 Качество модели 0.7039800995024875
Номер итерации: 6 Качество модели 0.7039800995024875
Номер итерации: 7 Качество модели 0.7039800995024875
Номер итерации: 8 Качество модели 0.7039800995024875
Номер итерации: 9 Качество модели 0.7039800995024875
Номер итерации: 10 Качество модели 0.7039800995024875
Номер итерации: 11 Качество модели 0.7039800995024875
Номер итерации: 12 Качество модели 0.7039800995024875
Номер итерации: 13 Качество модели 0.7039800995024875
Номер итерации: 14 Качество модели 0.7039800995024875
Номер итерации: 15 Качество модели 0.7039800995024875
Номер итерации: 16 Качество модели 0.7039800995024875
Номер итерации: 17 Качество модели 0.7039800995024875
Номер итерации: 18 Качество модели 0.7039800995024875
Номер итерации: 19 Качество модели 0.

### Показатели точности для трех моделей

In [20]:
print('Дерево решений:', round(best_result_dr, 3), "при глубине:", best_depth)
print('Случайный лес:', round(best_result_sl, 3), "при количестве деревьев:", best_est)
print('Логистическая регрессия:', round(best_result_lr, 3), "при количестве итераций:", itera )

Дерево решений: 0.799 при глубине: 9
Случайный лес: 0.794 при количестве деревьев: 26
Логистическая регрессия: 0.709 при количестве итераций: 100


***Вывод***

В ходе исследования были получены следующие данные:

* Точность модели дерева решения при глубине 9 равна 0.799. Время выполнения кода и обучения 0,345 секунд.
* Точность модели случайного леса при количестве деревьев 18 равна 0.81. Время выполнения кода и обучения 2,32 секунд.
* Точность модели логистической регресии равна 0.762. Время выполнения кода и обучения 32,1 секунд.

## Проверьте модель на тестовой выборке

### Дерево решения на тестовой выборке

In [21]:
test_prediction_dr = best_model_dr.predict(features_test)
test_result_dr = accuracy_score(target_test, test_prediction_dr)
print('Точность модели дерева решения на тестовой выборке', round(test_result_dr, 3))

Точность модели дерева решения на тестовой выборке 0.779


### Случайный лес на тестовой выборке

In [22]:
test_prediction_sl = best_model_sl.predict(features_test)
test_result_sl = accuracy_score(target_test, test_prediction_sl)
print('Точность модели случайного леса на тестовой выборке', round(test_result_sl, 3))

Точность модели случайного леса на тестовой выборке 0.786


### Логистическая регрессия на тестовой выборке

In [23]:
test_prediction_lr = best_model_lr.predict(features_test)
test_result_lr = accuracy_score(target_test, test_prediction_lr)
print('Точность модели логистической регрессии на тестовой выборке', round(test_result_lr, 3))

Точность модели логистической регрессии на тестовой выборке 0.706


**Вывод**

Учитывая, что мы хотим добиться точности 0.75, то мы можем использовать либо "Дерево решений", либо "Случайный лес". 

Для наиболее качественных результаты рекомендуем использовать модель Случайного леса, т.к. на тестовой выборке показал наилучший результат: 0.786 и на исследуемой модели: 0.794

## Итоговый вывод

В ходе проделанной работы было выполнено:

* Изучен файл с данными.
* Разбиты данные на три выборки: обучающая, валидационная и тестовая.
* Были исследованны три модели: Дерево решений, Случайный лес и Логистическая регрессия.
    * Точность модели дерева решения при глубине 9 равна 0.799. Время выполнения кода и обучения 0,345 секунд.
    * Точность модели случайного леса при количестве деревьев 18 равна 0.81. Время выполнения кода и обучения 2,32 секунд.
    * Точность модели логистической регресии равна 0.762. Время выполнения кода и обучения 32,1 секунд.
* Найдены оптимальные параметры для каждой модели и выбрана одна из них для обучения модели.
    * Точность модели случайного леса для тестовой выборки: 0.786